In [ ]:
pip install google-api-python-client

In [ ]:
import googleapiclient.discovery
import pandas as pd
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
API_KEY = os.getenv("Youtube_API_KEY")

youtube = build('youtube', 'v3', developerKey=API_KEY)

In [ ]:
def searching(query, number):
    request = youtube.search().list(
        part='id',
        maxResults= number,
        q=query,
        order = 'relevance',
        type ='video',
        fields='items/id'
        )

    response = request.execute()
    
    id_list = []

    for item in response['items']:
        id_list.append(item['id']['videoId'])
    
    return id_list

In [ ]:
def videoinfo(id_list):
    video = []
    for i in id_list: 
        request = youtube.videos().list(
            part="snippet, statistics",
            id= i
        )

        response = request.execute()

        each_snippet = response['items'][0]['snippet']
        each_stat = response['items'][0]["statistics"]

        video.append([
            i,
            each_snippet['publishedAt'],
            each_snippet["title"],
            each_snippet["channelTitle"],
            each_stat["viewCount"],
            each_stat["commentCount"] #the number of comments of a video, does not have replies count

        ])

    df = pd.DataFrame(video, 
                       columns=['video_id', 'published_at', 'video_title', 'publisher','view_count','comment_count'])
    return df

In [ ]:
def getcomments(video):
  request = youtube.commentThreads().list(
      part="snippet",
      videoId=video,
      maxResults=100
  )

  comments = []

  # Execute the request.
  response = request.execute()

  # Get the comments from the response.
  for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['textOriginal'],
          comment['videoId'],
      ])

  while (1 == 1):
    try:
     nextPageToken = response['nextPageToken']
    except KeyError:
     break
    nextPageToken = response['nextPageToken']
    # Create a new request object with the next page token.
    nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=100, pageToken=nextPageToken)
    # Execute the next request.
    response = nextRequest.execute()
    # Get the comments from the next response.
    
    
    for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['textOriginal'],
          comment['videoId'],
      ])

  df = pd.DataFrame(comments, columns=['AUTHOR', 'DATE', 'TEXT', 'video_id'])
  return df

In [ ]:
Query = 'elon musk acquisition of twitter'
Number = 3

Videos = searching(Query, Number)

In [ ]:
Video_Details = videoinfo(Videos)
Video_Details

In [ ]:
Comments = pd.DataFrame()
for i in Videos:
#for i in['SKSsnsAEcn8']:
  com = getcomments(i)
  Comments = pd.concat([Comments, com])

Comments

In [ ]:
Comments['video_id'].value_counts()

In [ ]:
Comments.to_csv("Youtube Comments.csv")